In [1]:
import os
import sys

sys.path.append('..')

import numpy as np
import pandas as pd
import scipy.optimize as opt
from pandas import DataFrame

from Constants.constants import Constants
from Constants.parameters import Parameters as parameters
from Utilities import generate_profitability_distribution, get_range, inter_product

In [2]:
class FirmValue(Constants):
    """
    This function used to find the optimal value of firm value given
    """

    def __init__(self, delta=None, rho=None, mu=None, gamma=None, theta=None, sigma=None, lambda_=None):
        """
        init

        :param delta: depreciation rate.
        :param rho: autocorrelation of profitability shock,
        :param mu: drift of profitability shock
        :param gamma: adjustment cost of investment.
        :param theta: collateral parameter
        :param sigma: std. dev. of profitability shocks,
        :param lambda_: equity issuance cost.
        :return: value function and firm
        """
        self._delta = None
        self._rho = None
        self._mu = None
        self._gamma = None
        self._theta = None
        self._sigma = None
        self._lambda = None
        self._profitability_grid = None
        self._transition_matrix = None
        self._investment_grid = None
        self._debt_grid = None
        self._debt_prime_grid = None

        self._firm_value = None

        # store policy matrix
        self._debt_policy_matrix = None
        self._invest_policy_matrix = None

        self.initialize(delta, rho, mu, gamma, theta, sigma, lambda_)

    def initialize(self, delta=None, rho=None, mu=None, gamma=None, theta=None, sigma=None, lambda_=None):
        self._delta = delta if delta is not None else parameters.delta_
        self._rho = rho if rho is not None else parameters.rho_
        self._mu = mu if mu is not None else parameters.mu_
        self._gamma = gamma if gamma is not None else parameters.gamma_
        self._theta = theta if theta is not None else parameters.theta_
        self._sigma = sigma if sigma is not None else parameters.sigma_
        self._lambda = lambda_ if lambda_ is not None else parameters.lambda_
        self._profitability_grid, self._transition_matrix = generate_profitability_distribution(self._mu, self._rho,
                                                                                                self._sigma, self.Z_NUM)

        self._debt_grid = get_range(-self._theta, self._theta, self.P_NUM)
        self._investment_grid = get_range(self._delta * (2 - np.ceil(self.I_NUM / (2 * self.DELP))),
                                          self._delta * (2 + np.ceil(self.I_NUM / (2 * self.DELP))),
                                          self.I_NUM)
        self._debt_prime_grid = get_range(-self._theta, self._theta, self.P_NEXT_NUM)
        self._firm_value = np.zeros((self.P_NUM, self.Z_NUM))
        self._debt_policy_matrix = np.zeros((self.P_NUM, self.Z_NUM), dtype=int)
        self._invest_policy_matrix = np.zeros((self.P_NUM, self.Z_NUM), dtype=int)

    def optimize(self):
        """
        Optimize model from BKW 2018 crs_toni_v2.f90
        :return: error code list
            0: Succeed
            1: Don't coverage
            2: Too large model difference
        """
        firm_value = self._firm_value.copy()
        all_val = np.zeros((self.P_NUM, self.P_NEXT_NUM, self.I_NUM, self.Z_NUM))
        queue_i = np.zeros((self.P_NEXT_NUM, self.I_NUM, self.Z_NUM))
        payoff = self.get_payoff_matrix()

        for _ in range(self.MAX_ITERATION):
            expected_fv = np.dot(firm_value, self._transition_matrix)
            expected_fv_prime = np.zeros((self.P_NEXT_NUM, self.Z_NUM))
            for ip in range(self.P_NEXT_NUM):
                ipd = int(ip * (self.P_NUM - 1) / (self.P_NEXT_NUM - 1))
                ipu = ipd + 1

                if ipu >= self.P_NUM:
                    ipd = self.P_NUM - 1
                    expected_fv_prime[ip, :] = expected_fv[ipd, :]
                else:
                    frac = (self._debt_grid[ipu] - self._debt_prime_grid[ip]) / (
                                self._debt_grid[ipu] - self._debt_grid[ipd])
                    # print(frac)
                    expected_fv_prime[ip, :] = expected_fv[ipd, :] * frac + expected_fv[ipu, :] * (1 - frac)

            for i in range(self.I_NUM):
                queue_i[:, i, :] = expected_fv_prime * (1 - self._delta + self._investment_grid[i])

            for ip in range(self.P_NUM):
                all_val[ip, :, :, :] = payoff[ip, :, :, :] + self.BETA * queue_i

            firm_value_next = np.max(np.max(all_val, axis=1), axis=1)
            difference = firm_value_next - firm_value
            model_diff = np.max(np.abs(difference))
            if _ % 100 == 0:
                print('Iteration: %d, Model difference: %f' % (_, model_diff))
            if model_diff < self.COVERAGE_THRESHOLD:
                break
            elif model_diff > self.DIFF_MAX_THRESHOLD:
                return 2
                # break

            firm_value = firm_value_next.copy()

        else:
            return 1

        self._firm_value = firm_value.copy()

        for iz in range(self.Z_NUM):
            for ip in range(self.P_NUM):
                current_value_firm = all_val[ip, :, :, iz]
                debt_max_i, invest_max_i = np.unravel_index(np.argmax(current_value_firm, axis=None),
                                                            current_value_firm.shape)
                self._debt_policy_matrix[ip, iz] = debt_max_i
                self._invest_policy_matrix[ip, iz] = invest_max_i
                return 0

    def optimize_terry(self):
        """
        Follow Stephen James Terry. Junior Accounting Theory Workshop, intro to structural estimation, 2018

        :return: same as optimize
        """
        debt_policy_matrix = self._debt_policy_matrix.copy()
        invest_policy_matrix = self._invest_policy_matrix.copy()
        firm_value = self._firm_value.copy()
        firm_value_last = firm_value.copy()

        payoff = self.get_payoff_matrix()

        for _ in range(self.MAX_ITERATION):
            # value function iteration
            for __ in range(self.VALUE_FUNCTION_ITERATION):
                for ip in range(self.P_NUM):
                    for iz in range(self.Z_NUM):
                        invest = self._investment_grid[invest_policy_matrix[ip, iz]]
                        debt_prime = self._debt_prime_grid[debt_policy_matrix[ip, iz]]
                        cfrac, debt_low_i, debt_up_i = inter_product(debt_prime, self._debt_grid)
                        fv_prime = firm_value[debt_low_i, :] * cfrac + firm_value[debt_up_i, :] * (1 - cfrac)
                        current_payoff = payoff[ip, debt_policy_matrix[ip, iz], invest_policy_matrix[ip, iz], iz]
                        firm_value[ip, iz] = current_payoff + self.BETA * np.dot(
                            fv_prime, self._transition_matrix[iz, :]) * (1 - self._delta + invest)

            # update policy matrix
            for ip in range(self.P_NUM):
                for iz in range(self.Z_NUM):
                    firm_value_prob = np.zeros((self.P_NEXT_NUM, self.I_NUM))
                    for ip_prime in range(self.P_NEXT_NUM):
                        for ii in range(self.I_NUM):
                            debt_prime = self._debt_prime_grid[ip_prime]

                            cfrac, debt_low_i, debt_up_i = inter_product(debt_prime, self._debt_grid)
                            fv_prime = firm_value[debt_low_i, :] * cfrac + firm_value[debt_up_i, :] * (1 - cfrac)
                            firm_value_prob[ip_prime, ii] = payoff[ip, ip_prime, ii, iz] + self.BETA * np.dot(
                                fv_prime, self._transition_matrix[iz, :]) * (1 - self._delta + self._investment_grid[
                                ii])

                    debt_max_i, invest_max_i = np.unravel_index(np.argmax(firm_value_prob, axis=None),
                                                                firm_value_prob.shape)
                    debt_policy_matrix[ip, iz] = debt_max_i
                    invest_policy_matrix[ip, iz] = invest_max_i

            # check difference
            value_difference = firm_value - firm_value_last
            diff = np.max(np.abs(value_difference))
            if _ % 10 == 0:
                print('Iteration: %d, Model difference: %f' % (_, diff))
            if diff > self.DIFF_MAX_THRESHOLD:
                return 2
            elif diff < self.COVERAGE_THRESHOLD:
                self._firm_value = firm_value.copy()
                self._debt_policy_matrix = debt_policy_matrix.copy()
                self._invest_policy_matrix = invest_policy_matrix.copy()
                return 0

            firm_value_last = firm_value.copy()

        else:
            # not coverage
            return 1

    def set_model_parameters(self, delta=None, rho=None, mu=None, gamma=None, theta=None, sigma=None, lambda_=None):
        if delta is not None:
            self.set_delta(delta)

        if rho is not None:
            self.set_rho(rho)

        if mu is not None:
            self.set_mu(mu)

        if gamma is not None:
            self.set_gamma(gamma)

        if theta is not None:
            self.set_theta(theta)

        if sigma is not None:
            self.set_sigma(sigma)

        if lambda_ is not None:
            self.set_lambda(lambda_)

    def set_delta(self, delta):
        self._delta = delta

    def set_rho(self, rho):
        self._rho = rho

    def set_mu(self, mu):
        self._mu = mu

    def set_gamma(self, gamma):
        self._gamma = gamma

    def set_theta(self, theta):
        self._theta = theta

    def set_sigma(self, sigma):
        self._sigma = sigma

    def set_lambda(self, lambda_):
        self._lambda = lambda_

    def get_payoff_matrix(self):
        payoff = np.zeros((self.P_NUM, self.P_NEXT_NUM, self.I_NUM, self.Z_NUM))

        for ip in range(self.P_NUM):
            for ip_prime in range(self.P_NEXT_NUM):
                for ii in range(self.I_NUM):
                    for iz in range(self.Z_NUM):
                        payoff[ip, ip_prime, ii, iz] = self.get_payoff(
                            self._profitability_grid[iz], self._investment_grid[ii], self._debt_grid[ip],
                            self._debt_prime_grid[ip_prime])
        return payoff

    def get_payoff(self, profitability, investment, debt, next_debt):
        payoff = profitability - investment - 0.5 * self._gamma * (investment ** 2) - debt * (1 + self.RF) \
                 + next_debt * (1 - self._delta + investment)

        if payoff < 0:
            payoff *= (1 + self._lambda)

        return payoff

    def simulate_model(self, years, firms):
        """
        simulate model
        :param years: number of year
        :param firms: number of firms
        :return: simulated model vectors
            firm_id
            year
            value
            profitability
            debt
            investment
            payoff
        """
        # initialize
        init_value = np.random.random(firms)
        profit_index = [int(i * self.Z_NUM) for i in init_value]
        debt_index = [int(i * self.P_NUM) for i in init_value]
        value_array = [self._firm_value[debt_index[i], profit_index[i]] for i in range(firms)]
        investment_array = [self._investment_grid[self._invest_policy_matrix[debt_index[i], profit_index[i]]] for i in
                            range(firms)]
        debt_array = [self._debt_grid[int(i * self.P_NUM)] for i in init_value]
        trans_cdf = self._transition_matrix.copy()

        for i in range(self.Z_NUM - 2):
            trans_cdf[:, i + 1] += trans_cdf[:, i]

        trans_cdf[:, self.Z_NUM - 1] = 1

        # run simulation
        simulated_data_list = list()
        for year_i in range(years):
            simulated_data = DataFrame(
                columns=['value', 'profitability', 'debt', 'investment', 'payoff', 'debt_prime'],
                index=list(range(firms)))

            simulated_data.loc[:, 'profitability'] = self._profitability_grid[profit_index]
            simulated_data.loc[:, 'debt'] = debt_array
            simulated_data.loc[:, 'value'] = value_array

            # get next period profitability
            next_shock = np.random.random(firms)
            for i in range(firms):
                # save current data
                profitability = simulated_data.loc[i, 'profitability']
                debt = simulated_data.loc[i, 'debt']

                # determine next period values
                cfrac, low_index, up_index = inter_product(debt_array[i], self._debt_grid)
                profit_index[i] = len(trans_cdf[profit_index[i]][trans_cdf[profit_index[i]] < next_shock[i]])
                debt_prime_up = self._debt_prime_grid[self._debt_policy_matrix[up_index, profit_index[i]]]
                debt_prime_low = self._debt_prime_grid[self._debt_policy_matrix[low_index, profit_index[i]]]
                invest_prime_up = self._investment_grid[self._invest_policy_matrix[up_index, profit_index[i]]]
                invest_prime_low = self._investment_grid[self._invest_policy_matrix[low_index, profit_index[i]]]

                debt_array[i] = cfrac * debt_prime_low + (1 - cfrac) * debt_prime_up
                value_array[i] = cfrac * self._firm_value[low_index, profit_index[i]] + (1 - cfrac) \
                                 * self._firm_value[up_index, profit_index[i]]
                investment_array[i] = cfrac * invest_prime_low + (1 - cfrac) * invest_prime_up

                simulated_data.loc[i, 'payoff'] = self.get_payoff(profitability, investment_array[i], debt,
                                                                  debt_array[i])

            simulated_data.loc[:, 'year'] = year_i
            simulated_data.loc[:, 'firm_id'] = list(range(firms))
            simulated_data.loc[:, 'debt_prime'] = debt_array
            simulated_data.loc[:, 'investment'] = investment_array
            simulated_data_list.append(simulated_data)

        return pd.concat(simulated_data_list, ignore_index=True, sort=False)


In [6]:
cov_matrix_list = ''' 1.109708117	-0.005453733	0.057355899	0.004006414	-0.011096374	-0.007197551	0.005645	-0.009212411
-0.005453733	0.017286135	-0.000443486	0.000297896	-0.002858936	0.001057859	-0.003144528	0.001290623
0.057355899	-0.000443486	0.050829381	0.003399744	-0.005320978	0.000654373	0.025193349	0.000681779
0.004006414	0.000297896	0.003399744	0.000421314	-0.000845842	0.00012831	0.000718566	0.000148138
-0.011096374	-0.002858936	-0.005320978	-0.000845842	0.033832022	-0.003816715	0.037418057	-0.001367266
-0.007197551	0.001057859	0.000654373	0.00012831	-0.003816715	0.001233236	-0.002444826	0.000397526
0.005645	-0.003144528	0.025193349	0.000718566	0.037418057	-0.002444826	0.121059595	-0.000328985
-0.009212411	0.001290623	0.000681779	0.000148138	-0.001367266	0.000397526	-0.000328985	0.001289656'''.split('\n')
cov_matrix = np.zeros((8, 8))

for i, line in enumerate(cov_matrix_list):
    for j, cell in enumerate(line.split('\t')):
        cov_matrix[i, j] = float(cell)

In [8]:

def get_moments(fv: FirmValue, process_num:int=1):
    """
    Simulate model and return moments
    :param fv: an optimized firm value
    :param process_num: number of multiprocessing number
    :return: a list [mean investment, var investment, mean leverage, var leverage, mean payoff, var payoff,
                     mean profit, var profit]
        0.0768111297195329
        0.0032904184631855
        0.1885677166674841
        0.0285271524764669
        0.0012114713963756
        0.0058249053810193
        0.1421154126428439
        0.0080642043112130

    """
    if process_num > 1:
        firm_num = FirmValue.N_FIRMS // process_num
        firm_num_reminder = firm_num % process_num
        firm_num_list = [firm_num for _ in range(process_num)]

        for i in range(firm_num_reminder):
            firm_num_list[i] += 1

        pool = pathos.multiprocessing.ProcessPool(process_num)
        simulated_result = pool.map(lambda x: fv.simulate_model(FirmValue.N_YEARS, x), firm_num_list)
        simulated_result2 = list()

        for i, result_df in enumerate(simulated_result):
            result_df.loc[:, 'firm_id'] = result_df['firm_id'].apply(lambda x: '{}_{}'.format(i, x))
            simulated_result2.append(result_df)
        sim_data: DataFrame = pd.concat(simulated_result2, ignore_index=True, sort=False)

    else:
        sim_data: DataFrame = fv.simulate_model(years=FirmValue.N_YEARS, firms=9000)
    sim_data_valid: DataFrame = sim_data.iloc[-8:].copy()
    result_series = np.zeros(8)
    mean_data = sim_data_valid.mean()
    var_data = sim_data_valid.var()
    result_series[0] = mean_data['debt']
    result_series[1] = var_data['debt']
    result_series[1] = mean_data['investment']
    result_series[2] = var_data['investment']
    result_series[4] = mean_data['payoff']
    result_series[5] = var_data['payoff']
    result_series[6] = mean_data['profitability']
    result_series[7] = var_data['profitability']

    return result_series


def get_moments_error(data_mom, sim_mom, weighted_matrix):
    err_mom = data_mom - sim_mom
    crit_val = err_mom.T @ weighted_matrix @ err_mom
    return crit_val


def criterion(params, *args):
    mu, rho, sigma, delta, gamma, theta, lambda_ = params
    process_num: int = args[0]
    fv = FirmValue(delta=delta, mu=mu, rho=rho, sigma=sigma, theta=theta, lambda_=lambda_, gamma=gamma)
    error_code = fv.optimize_terry()
    if error_code != 0:
        return 1e18
    data_moments = np.array([0.1885677166674841, 0.0285271524764669, 0.0768111297195329, 0.0032904184631855, 
                             0.0012114713963756, 0.0058249053810193, 0.1421154126428439, 0.0080642043112130])
    sim_moments = get_moments(fv, process_num)
    moments_error = get_moments_error(data_moments, sim_moments, cov_matrix)
    print('Moments errors are:', moments_error)
    return moments_error

Result for np.eye as weighted matrix

In [ ]:
    params_init_1 = np.array([-2.2067, 0.8349, 0.3594, 0.044, 29.966, 0.381, 0.182])
    results1_1 = opt.minimize(criterion, params_init_1, args=(1,), method='L-BFGS-B',
                              options={'eps': 1e-1, 'gtol': 1e-3},
                              bounds=(
                                  (-6.5, -0.5), (0.3, 0.9), (0.05, 0.6), (0.01, 0.2), (3, 30), (0.1, 0.7),
                                  (0.01, 0.25)))
    print(results1_1)

Iteration: 0, Model difference: 3.130944
Iteration: 10, Model difference: 0.308268
Iteration: 0, Model difference: 3.111329
Iteration: 0, Model difference: 3.077723
Iteration: 10, Model difference: 0.000034
Moments errors are: 0.01882599284924767
Iteration: 0, Model difference: 3.156272
Iteration: 0, Model difference: 12.528848
Moments errors are: 0.049580405144583166
Iteration: 0, Model difference: 3.118188
Iteration: 10, Model difference: 0.508150
Iteration: 0, Model difference: 3.252004
Iteration: 10, Model difference: 0.436676
Iteration: 0, Model difference: 2.866059
Iteration: 10, Model difference: 70537.613236
Iteration: 0, Model difference: 17.736477
Moments errors are: 0.0390004950176372
Iteration: 0, Model difference: 17.733129
Moments errors are: 0.04059442906935675
Iteration: 0, Model difference: 17.735611
Moments errors are: 0.034930581915394554
Iteration: 0, Model difference: 17.746830
Moments errors are: 0.031836888904092836
Iteration: 0, Model difference: 8.370495
Moment

In [ ]:
    params_init_1 = np.array([-2.2067, 0.8349, 0.3594, 0.0449, 29.9661, 0.3816, 0.1829])
    results1_1 = opt.dual_annealing(criterion, args=(1,), x0=params_init_1,
                              bounds=(
                                  (-3.5, -1.5), (0.8, 0.9), (0.3, 0.4), (0.01, 0.05), (25, 30), (0.1, 0.4),
                                  (0.01, 0.25)))
    print(results1_1)

Iteration: 0, Model difference: 2.108681
Iteration: 0, Model difference: 2.548648
Iteration: 0, Model difference: 2.430482
Moments errors are: 0.001761823992359608
Iteration: 0, Model difference: 2.194735
Iteration: 0, Model difference: 2.253752
Moments errors are: 0.03663644568876667
Iteration: 0, Model difference: 0.926258
Iteration: 10, Model difference: 0.001140
Moments errors are: 0.0064888481683688555
Iteration: 0, Model difference: 1.684006
Iteration: 0, Model difference: 1.606390
Iteration: 0, Model difference: 2.361356
Iteration: 10, Model difference: 0.000027
Moments errors are: 0.0006519275233890279
Iteration: 0, Model difference: 2.374607
Iteration: 10, Model difference: 0.002030
Moments errors are: 0.003815995344177767
Iteration: 0, Model difference: 2.360482
Iteration: 10, Model difference: 0.000028
Moments errors are: 0.0008622770467347481
Iteration: 0, Model difference: 1.451297
Iteration: 0, Model difference: 1.943349
Iteration: 10, Model difference: 4.037827
Iteration

Iteration: 0, Model difference: 2.228674
Moments errors are: 0.0008921507778102685
Iteration: 0, Model difference: 2.228674
Moments errors are: 0.0006868818863804406
Iteration: 0, Model difference: 2.228674
Moments errors are: 0.0008538280855897667
Iteration: 0, Model difference: 2.228674
Moments errors are: 0.0006519611822694925
Iteration: 0, Model difference: 2.303495
Iteration: 10, Model difference: 0.000020
Moments errors are: 0.0013496564249757064
Iteration: 0, Model difference: 2.303495
Iteration: 10, Model difference: 0.000020
Moments errors are: 0.001195364224218154
Iteration: 0, Model difference: 2.303495
Iteration: 10, Model difference: 0.000020
Moments errors are: 0.000925444394992513
Iteration: 0, Model difference: 2.303495
Iteration: 10, Model difference: 0.000020
Moments errors are: 0.0014877214848888157
Iteration: 0, Model difference: 2.303495
Iteration: 10, Model difference: 0.000020
Moments errors are: 0.0012006970363758467
Iteration: 0, Model difference: 2.303495
Iter

In [ ]:
results1_1

Result for cov matrix in BKW 2018 as weighted matrix

In [ ]:
    params_init_1 = np.array([-2.2067, 0.8349, 0.3594, 0.044, 29.966, 0.381, 0.182])
    results1_2 = opt.minimize(criterion, params_init_1, args=(1,), method='L-BFGS-B',
                              options={'eps': 1e-1, 'gtol': 1e-3},
                              bounds=(
                                  (-6.5, -0.5), (0.3, 0.9), (0.05, 0.6), (0.01, 0.2), (3, 30), (0.1, 0.7),
                                  (0.01, 0.25)))
    print(results1_2)

Iteration: 0, Model difference: 3.130944
Iteration: 10, Model difference: 0.308268
Iteration: 0, Model difference: 3.111329
Iteration: 0, Model difference: 3.077723
Iteration: 10, Model difference: 0.000034
Moments errors are: 0.016771672949390245
Iteration: 0, Model difference: 3.156272
Iteration: 0, Model difference: 12.528848
Moments errors are: 0.031018880582208103
Iteration: 0, Model difference: 3.118188
Iteration: 10, Model difference: 0.508150
Iteration: 0, Model difference: 3.252004
Iteration: 10, Model difference: 0.436676
Iteration: 0, Model difference: 2.866059
Iteration: 10, Model difference: 70537.613236
Iteration: 0, Model difference: 17.736477
Moments errors are: 0.024313302564696448
Iteration: 0, Model difference: 17.733129
Moments errors are: 0.027192608487342486
Iteration: 0, Model difference: 17.735611
Moments errors are: 0.02233566741017792
Iteration: 0, Model difference: 17.746830
Moments errors are: 0.019846782532072397
Iteration: 0, Model difference: 8.370495
Mom

Iteration: 0, Model difference: 17.571758
Moments errors are: 0.0007946892470205792
Iteration: 0, Model difference: 7.867949
Moments errors are: 0.0010239514820474323
Iteration: 0, Model difference: 17.501137
Moments errors are: 0.0007786537768895155
Iteration: 0, Model difference: 17.687273
Moments errors are: 0.006123246574098324
Iteration: 0, Model difference: 16.126684
Moments errors are: 0.0007786186507119242
Iteration: 0, Model difference: 17.570543
Moments errors are: 0.0008523331525028559
Iteration: 0, Model difference: 17.567586
Moments errors are: 0.0008324248674834835
Iteration: 0, Model difference: 17.569851
Moments errors are: 0.0009043028552571421
Iteration: 0, Model difference: 17.579723
Moments errors are: 0.000988692922511752
Iteration: 0, Model difference: 7.876670
Moments errors are: 0.0007830752854431628
Iteration: 0, Model difference: 17.509078
Moments errors are: 0.0008096653096445036
Iteration: 0, Model difference: 17.695228
Moments errors are: 0.0059131200221159

In [33]:
results1_2

      fun: 0.0009172056112549222
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00018294, -0.00146537,  0.00012174,  0.00087829,  0.00141824,
        0.06196424,  0.0014405 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 200
      nit: 6
     njev: 25
   status: 0
  success: True
        x: array([-2.24648901,  0.32716889,  0.41296885,  0.19433193, 29.97242835,
        0.18486076,  0.22302292])

In [4]:
    params_init_1 = np.array([-2.2067, 0.8349, 0.3594, 0.044, 29.966, 0.381, 0.182])
    mu, rho, sigma, delta, gamma, theta, lambda_ = params_init_1
    fv = FirmValue(delta=delta, mu=mu, rho=rho, sigma=sigma, theta=theta, lambda_=lambda_, gamma=gamma)
    error_code = fv.optimize_terry()

Iteration: 0, Model difference: 3.130944
Iteration: 10, Model difference: 0.308268


In [ ]:
    data_moments = np.array([0.1885677166674841, 0.0285271524764669, 0.0768111297195329, 0.0032904184631855, 
                             0.0012114713963756, 0.0058249053810193, 0.1421154126428439, 0.0080642043112130])
    sim_moments = get_moments(fv, 1)
    moments_error = get_moments_error(data_moments, sim_moments, cov_matrix)

In [ ]:
sim_moments